# 小盘成长三因子选股
一、买入逻辑
每周根据三因子选股获取target list，并有条件的剔除部分标的
1.三因子：1）5年营业收入增长率SG，取大值，选取范围见代码，2）成长综合因子MS，取大值,因子占比见代码,3）PEG，换手波动率，取小值，选取范围见代码
2.三因子均取值范围为流通市值最小的5只证券
3.需剔除的标的：1）eps<0,2）最近20个交易日内有涨停的股票和最近20天持有过的证券的交集进入黑名单，此处选近20个交易日，或近一个月，3）剔除当日涨停、跌停、停牌、科创板、ST

二、卖出逻辑
1.每交易日下午14：00查看证券前一交易日是否涨停，若今天没有继续涨停就卖出，但如果涨停即使不在应买入列表仍暂时持有
2.每周选股，未被再次选入的证券卖出，但如果今日未入选的标的昨日涨停，则继续持有



In [ ]:
#导入函数库
from jqdata import *
from jqfactor import get_factor_values
from jqlib.technical_analysis import *
import numpy as np
import pandas as pd
import statsmodels.api as sm
import datetime as dt


#初始化函数
def initialize(context):
    # 设定基准
    set_benchmark('000905.XSHG')
    # 用真实价格交易
    set_option('use_real_price', True)
    # 打开防未来函数
    set_option("avoid_future_data", True)
    # 将滑点设置为0
    set_slippage(FixedSlippage(0))
    # 设置交易成本万分之三，不同滑点影响可在归因分析中查看
    set_order_cost(OrderCost(open_tax=0, close_tax=0.001, open_commission=0.0003, close_commission=0.0003,
                             close_today_commission=0, min_commission=5), type='fund')
    # 过滤order中低于error级别的日志
    log.set_level('order', 'error')
    #初始化全局变量
    g.stock_num = 10
    g.limit_up_list = []  #记录持仓中上一个交易日涨停的股票
    g.hold_list = []  #当前持仓的全部股票
    g.history_hold_list = []  #过去一段时间内持仓过的股票
    g.not_buy_again_list = []  #最近买过且涨停过的股票一段时间内不再买入
    g.limit_days = 20  #不再买入的时间段天数
    g.target_list = []  #开盘前预操作股票池
    # 设置交易运行时间
    run_daily(prepare_stock_list, time='9:05', reference_security='000300.XSHG')
    run_weekly(weekly_adjustment, weekday=1, time='9:30', reference_security='000300.XSHG')
    run_daily(check_limit_up, time='14:00', reference_security='000300.XSHG')  #检查持仓中的涨停股是否需要卖出，对昨日涨停股票观察到尾盘如不涨停则提前卖出，如果涨停即使不在应买入列表仍暂时持有
    run_daily(print_position_info, time='15:10', reference_security='000300.XSHG')


#1-1 选股模块，获取聚宽因子值
def get_factor_filter_list(context, stock_list, jqfactor, sort, p1, p2):
    yesterday = context.previous_date
    score_list = get_factor_values(stock_list, jqfactor, end_date=yesterday, count=1)[jqfactor].iloc[
        0].tolist()  # 获取score
    df = pd.DataFrame(columns=['code', 'score'])
    df['code'] = stock_list
    df['score'] = score_list
    df = df.dropna()
    df.sort_values(by='score', ascending=sort, inplace=True)  # 因子排序
    filter_list = list(df.code)[int(p1 * len(stock_list)):int(p2 * len(stock_list))]  # 取因子值分位数p1到p2的证券code
    return filter_list


#1-2 选股模块
def get_stock_list(context):
    yesterday = str(context.previous_date)
    initial_list = get_all_securities().index.tolist()  # 获取所有证券
    initial_list = filter_new_stock(context, initial_list)  # 剔除次新股
    initial_list = filter_kcb_stock(context, initial_list)  # 剔除可科创板
    initial_list = filter_st_stock(initial_list)  # 剔除ST
    #SG 5年营业收入增长率
    sg_list = get_factor_filter_list(context, initial_list, 'sales_growth', False, 0, 0.1)  # 取因子排序后的前10%的证券，此处由大到小排序
    q = query(valuation.code, valuation.circulating_market_cap, indicator.eps).filter(
        valuation.code.in_(sg_list)).order_by(valuation.circulating_market_cap.asc())  # 获取估值数据，并以流通市值排序
    df = get_fundamentals(q, date=yesterday)
    df = df[df['eps'] > 0]  #剔除eps<=0的证券
    sg_list = list(df.code)  # 获取营业收入增长因子筛选的证券
    #MS
    factor_values = get_factor_values(initial_list, [
        'operating_revenue_growth_rate',  #营业收入增长率
        'total_profit_growth_rate',  #利润总额增长率
        'net_profit_growth_rate',  #净利润增长率
        'earnings_growth',  #5年盈利增长率
    ], end_date=yesterday, count=1)  # 获取因子值
    df = pd.DataFrame(index=initial_list, columns=factor_values.keys())
    df['operating_revenue_growth_rate'] = list(factor_values['operating_revenue_growth_rate'].T.iloc[:, 0])
    df['total_profit_growth_rate'] = list(factor_values['total_profit_growth_rate'].T.iloc[:, 0])
    df['net_profit_growth_rate'] = list(factor_values['net_profit_growth_rate'].T.iloc[:, 0])
    df['earnings_growth'] = list(factor_values['earnings_growth'].T.iloc[:, 0])
    df['total_score'] = 0.1 * df['operating_revenue_growth_rate'] + 0.35 * df['total_profit_growth_rate'] + 0.15 * df[
        'net_profit_growth_rate'] + 0.4 * df['earnings_growth']  # 成长多因子综合打分
    df = df.sort_values(by=['total_score'], ascending=False)
    complex_growth_list = list(df.index)[:int(0.1 * len(list(df.index)))]  # 获取成长综合因子打分前10%的证券
    q = query(valuation.code, valuation.circulating_market_cap, indicator.eps).filter(
        valuation.code.in_(complex_growth_list)).order_by(valuation.circulating_market_cap.asc())  # 获取估值数据，并以流通市值排序
    df = get_fundamentals(q)
    df = df[df['eps'] > 0]  #剔除eps<=0的证券
    ms_list = list(df.code)
    #PEG
    peg_list = get_factor_filter_list(context, initial_list, 'PEG', True, 0, 0.2)  # 获取PEG因子打分前20%的证券，此处由小到大排序
    turnover_list = get_factor_filter_list(context, peg_list, 'turnover_volatility', True, 0,
                                           0.5)  # 获取换手波因子分位数0到50%的证券code，由小到大排序
    q = query(valuation.code, valuation.circulating_market_cap, indicator.eps).filter(
        valuation.code.in_(turnover_list)).order_by(valuation.circulating_market_cap.asc())  # 获取估值数据，并以流通市值排序
    df = get_fundamentals(q, date=yesterday)
    peg_list = list(df.code)

    final_list = [sg_list, ms_list, peg_list]
    return final_list


#1-3 准备股票池，每日9：05运行
def prepare_stock_list(context):
    # 1...2
    #获取已持有列表
    g.hold_list = []
    for position in list(context.portfolio.positions.values()):
        stock = position.security
        g.hold_list.append(stock)
    #获取最近一段时间持有过的股票列表
    g.history_hold_list.append(g.hold_list)
    if len(g.history_hold_list) >= g.limit_days:  # 仅保留g.limit_days天内的
        g.history_hold_list = g.history_hold_list[-g.limit_days:]
    temp_set = set()
    for hold_list in g.history_hold_list:
        for stock in hold_list:
            temp_set.add(stock)  # 去重
    g.not_buy_again_list = list(temp_set)  # g.limit_days天内的不再买入
    #获取昨日涨停列表
    if g.hold_list != []:  #如果持仓不为空
        df = get_price(g.hold_list, end_date=context.previous_date, frequency='daily', fields=['close', 'high_limit'],
                       count=1, panel=False, fill_paused=False)
        df = df[df['close'] == df['high_limit']]
        g.high_limit_list = list(df.code)
    else:
        g.high_limit_list = []


#1-5 整体调整持仓，每周一9：30运行
def weekly_adjustment(context):
    #1 #获取应买入列表
    all_list = get_stock_list(context)  #获取SG、MS、peg三个因子筛选出的证券list,每个list提取前50的证券（此前已根据流通市值由小到大排序）
    sg_list = all_list[0][:5]
    ms_list = all_list[1][:5]
    peg_list = all_list[2][:5]
    union_list = list(set(sg_list).union(set(ms_list)).union(set(peg_list)))  # 三因子选出的证券去重
    q = query(valuation.code, valuation.circulating_market_cap).filter(valuation.code.in_(union_list)).order_by(
        valuation.circulating_market_cap.asc())
    df = get_fundamentals(q)
    g.target_list = list(df.code)  #...2
    g.target_list = filter_paused_stock(g.target_list)  #剔除当天停牌证券
    g.target_list = filter_limitup_stock(context, g.target_list)  #剔除当天涨停证券
    g.target_list = filter_limitdown_stock(context, g.target_list)  #剔除当天跌停证券
    #过滤最近买过且涨停过的股票
    recent_limit_up_list = get_recent_limit_up_stock(context, g.target_list, g.limit_days)  # 获取最近N个交易日内有涨停的股票
    black_list = list(set(g.not_buy_again_list).intersection(set(recent_limit_up_list)))   # 最近N个交易日内有涨停的股票和最近买过的证券的交集进入黑名单
    g.target_list = [stock for stock in g.target_list if stock not in black_list]  # 剔除黑名单证券
    #截取不超过最大持仓数的股票量
    g.target_list = g.target_list[:min(g.stock_num, len(g.target_list))]  # 去除超量证券
    #调仓卖出
    for stock in g.hold_list:
        if (stock not in g.target_list) and (stock not in g.high_limit_list):  # 现持有证券不在新选的目标证券中且昨日未涨停，则卖出
            log.info("卖出[%s]" % (stock))
            position = context.portfolio.positions[stock]  # 获取持有证券的仓位
            close_position(position)
        else:
            log.info("已持有[%s]" % (stock))  # 若昨日涨停或继续入选则继续持有
    #调仓买入
    position_count = len(context.portfolio.positions)  # 现有持仓票数
    target_num = len(g.target_list)  # 目标买入证券数
    if target_num > position_count:
        value = context.portfolio.cash / (target_num - position_count)  # 现金平均分配
        for stock in g.target_list:
            if context.portfolio.positions[stock].total_amount == 0:
                if open_position(stock, value):  # 买入证券
                    if len(context.portfolio.positions) == target_num: # 是否达成买入目标
                        break


#1-6 调整昨日涨停股票
def check_limit_up(context):
    now_time = context.current_dt
    if g.high_limit_list != []:
        #对昨日涨停股票观察到尾盘如不涨停则提前卖出，如果涨停即使不在应买入列表仍暂时持有
        for stock in g.high_limit_list:
            current_data = get_price(stock, end_date=now_time, frequency='1m', fields=['close', 'high_limit'],
                                     skip_paused=False, fq='pre', count=1, panel=False, fill_paused=True)
            if current_data.iloc[0, 0] < current_data.iloc[0, 1]:
                log.info("[%s]涨停打开，卖出" % (stock))
                position = context.portfolio.positions[stock]
                close_position(position)
            else:
                log.info("[%s]涨停，继续持有" % (stock))


#2-1 过滤停牌股票
def filter_paused_stock(stock_list):
    current_data = get_current_data()
    return [stock for stock in stock_list if not current_data[stock].paused]


#2-2 过滤ST及其他具有退市标签的股票
def filter_st_stock(stock_list):
    current_data = get_current_data()
    return [stock for stock in stock_list
            if not current_data[stock].is_st
            and 'ST' not in current_data[stock].name
            and '*' not in current_data[stock].name
            and '退' not in current_data[stock].name]


#2-3 获取最近N个交易日内有涨停的股票
def get_recent_limit_up_stock(context, stock_list, recent_days):
    stat_date = context.previous_date
    new_list = []
    for stock in stock_list:
        df = get_price(stock, end_date=stat_date, frequency='daily', fields=['close', 'high_limit'], count=recent_days,
                       panel=False, fill_paused=False)
        df = df[df['close'] == df['high_limit']]
        if len(df) > 0:
            new_list.append(stock)
    return new_list


#2-4 过滤涨停的股票
def filter_limitup_stock(context, stock_list):
    last_prices = history(1, unit='1m', field='close', security_list=stock_list)
    current_data = get_current_data()
    return [stock for stock in stock_list if stock in context.portfolio.positions.keys()
            or last_prices[stock][-1] < current_data[stock].high_limit]


#2-5 过滤跌停的股票
def filter_limitdown_stock(context, stock_list):
    last_prices = history(1, unit='1m', field='close', security_list=stock_list)
    current_data = get_current_data()
    return [stock for stock in stock_list if stock in context.portfolio.positions.keys()
            or last_prices[stock][-1] > current_data[stock].low_limit]


#2-6 过滤科创板
def filter_kcb_stock(context, stock_list):
    return [stock for stock in stock_list if stock[0:3] != '688']


#2-7 过滤次新股
def filter_new_stock(context, stock_list):
    yesterday = context.previous_date
    return [stock for stock in stock_list if
            not yesterday - get_security_info(stock).start_date < datetime.timedelta(days=375)]


#3-1 交易模块-自定义下单
def order_target_value_(security, value):
    if value == 0:
        log.debug("Selling out %s" % (security))
    else:
        log.debug("Order %s to value %f" % (security, value))
    return order_target_value(security, value)


#3-2 交易模块-开仓
def open_position(security, value):
    order = order_target_value_(security, value)
    if order != None and order.filled > 0:
        return True
    return False


#3-3 交易模块-平仓
def close_position(position):
    security = position.security
    order = order_target_value_(security, 0)  # 可能会因停牌失败
    if order != None:
        if order.status == OrderStatus.held and order.filled == order.amount:
            return True
    return False


#3-4 交易模块-调仓
def adjust_position(context, buy_stocks, stock_num):
    for stock in context.portfolio.positions:
        if stock not in buy_stocks:
            log.info("[%s]不在应买入列表中" % (stock))
            position = context.portfolio.positions[stock]
            close_position(position)
        else:
            log.info("[%s]已经持有无需重复买入" % (stock))

    position_count = len(context.portfolio.positions)
    if stock_num > position_count:
        value = context.portfolio.cash / (stock_num - position_count)
        for stock in buy_stocks:
            if context.portfolio.positions[stock].total_amount == 0:
                if open_position(stock, value):
                    if len(context.portfolio.positions) == g.stock_num:
                        break


#4-1 打印每日持仓信息
def print_position_info(context):
    #打印当天成交记录
    trades = get_trades()
    for _trade in trades.values():
        print('成交记录：' + str(_trade))
    #打印账户信息
    for position in list(context.portfolio.positions.values()):
        securities = position.security
        cost = position.avg_cost
        price = position.price
        ret = 100 * (price / cost - 1)
        value = position.value
        amount = position.total_amount
        print('代码:{}'.format(securities))
        print('成本价:{}'.format(format(cost, '.2f')))
        print('现价:{}'.format(price))
        print('收益率:{}%'.format(format(ret, '.2f')))
        print('持仓(股):{}'.format(amount))
        print('市值:{}'.format(format(value, '.2f')))
        print('———————————————————————————————————')
    print('———————————————————————————————————————分割线————————————————————————————————————————')